# Configurare baza de date

In [1]:
!pip install neo4j pandas
from neo4j import GraphDatabase
URI = "neo4j+s://1d172e4b.databases.neo4j.io"
AUTH = ("neo4j", "bHKIzZeC8J1umROBENDWTHUMD3VuIcTXYPYiANT9Omo")
driver = GraphDatabase.driver(URI, auth=AUTH)
def run_query(query, parameters=None):
    """Funcție helper pentru rularea query-urilor"""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]
try:
    driver.verify_connectivity()
    print("✓ Conectare reușită la Neo4j!")
except Exception as e:
    print(f"✗ Eroare la conectare: {e}")

✓ Conectare reușită la Neo4j!


## Curatare baza de date

In [2]:
clear_query = "MATCH (n) DETACH DELETE n"
run_query(clear_query)
print("Baza de date curățată!")

Baza de date curățată!


# Schema bazei de date

### 1. Noduri (Node Types)

| Label | Proprietăți | Descriere |
| :--- | :--- | :--- |
| **User** | `id`, `name`, `role`, `languages`, `soft_skills`, `open_to_work` | Nodul central (Studenți, Angajați, Recruiteri) |
| **Company** | `name`, `industry`, `work_culture`, `city` | Organizații și companii angajatoare |
| **University** | `name`, `country`, `rank` | Instituții de învățământ (pentru rețeaua Alumni) |
| **Project** | `name`, `tech_stack`, `type` | Proiecte (Open Source, Hackathon, Personale) |
| **Certification** | `name`, `issuer`, `difficulty` | Acreditări profesionale (ex: AWS, PMP) |
| **Post** | `id`, `topics`, `timestamp` | Conținut social (articole, știri) pentru interese |

### 2. Relații (Relationship Types)

| Tip Relație | De la -> Către | Proprietăți Cheie | Descriere |
| :--- | :--- | :--- | :--- |
| **WORKS_AT** | `User` -> `Company` | `role`, `current`, `end_year` | Istoricul profesional sau jobul curent |
| **STUDIED_AT** | `User` -> `University` | `degree`, `year` | Conectează userul de Alma Mater |
| **CONTRIBUTED_TO**| `User` -> `Project` | `commits` | Arată implicarea tehnică într-un proiect |
| **EARNED** | `User` -> `Certification`| `year` | Validează competențele userului |
| **FOLLOWS** | `User` -> `User` | *(fără proprietăți)* | Relația socială directă (prietenie/follow) |
| **ENGAGED_WITH** | `User` -> `Post` | `type` (Like, Comment) | Interes implicit (fără follow direct) |

### 3. Constrângeri (Data Integrity Constraints)

| Tip Constrângere | Target (Nod.proprietate) | Descriere                                                                                               |
| :--- | :--- |:--------------------------------------------------------------------------------------------------------|
| **UNIQUENESS** | `User.id` | Asigură că fiecare utilizator are un identificator unic în sistem. |
| **UNIQUENESS** | `Company.name` | Garantează că o companie apare o singură dată în graf (previne duplicate ex: "Google" vs "google").     |
| **UNIQUENESS** | `University.name` | Asigură consistența rețelei de Alumni; toți absolvenții sunt legați de același nod unic.                |
| **UNIQUENESS** | `Post.id` | Identificator unic pentru conținut, necesar pentru a gestiona like-uri și comentarii corect.            |
| **EXISTENCE** | `User.name` | (Opțional) Impune ca orice nod creat cu eticheta `User` să aibă obligatoriu un nume completat.          |

### Constrangeri

In [3]:
constrangeri_queries = [
    "CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE",
    "CREATE CONSTRAINT company_name_unique IF NOT EXISTS FOR (c:Company) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT uni_name_unique IF NOT EXISTS FOR (u:University) REQUIRE u.name IS UNIQUE",
    "CREATE CONSTRAINT post_id_unique IF NOT EXISTS FOR (p:Post) REQUIRE p.id IS UNIQUE",
    "CREATE CONSTRAINT project_name_unique IF NOT EXISTS FOR (p:Project) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT cert_name_unique IF NOT EXISTS FOR (c:Certification) REQUIRE c.name IS UNIQUE"
]

for c in constrangeri_queries:
    try:
        run_query(c)
    except Exception as e:
        print(f"Constrângere deja existentă sau eroare: {e}")

print("Constrângeri și indecsi create!")

Constrângeri și indecsi create!


### Noduri

In [4]:
#COMPANII
companii = """
CREATE
  (c1:Company {name: 'Google', industry: 'Tech', work_culture: ['Innovative', 'Scale'], city: 'Bucharest'}),
  (c2:Company {name: 'UiPath', industry: 'Automation', work_culture: ['Fast-paced', 'Remote'], city: 'Bucharest'}),
  (c3:Company {name: 'Bitdefender', industry: 'Security', work_culture: ['Security', 'Engineering'], city: 'Cluj-Napoca'}),
  (c4:Company {name: 'BCR', industry: 'Finance', work_culture: ['Corporate', 'Stable'], city: 'Bucharest'}),
  (c5:Company {name: 'Revolut', industry: 'FinTech', work_culture: ['Hustle', 'Data'], city: 'Remote'})
"""
run_query(companii)
print("Companii create!")

Companii create!


In [5]:
#UNIVERSITĂȚI
univ = """
CREATE
  (uni1:University {name: 'Politehnica Bucuresti', country: 'RO', rank: 800}),
  (uni2:University {name: 'Universitatea Bucuresti', country: 'RO', rank: 900}),
  (uni3:University {name: 'UBB Cluj', country: 'RO', rank: 850}),
  (uni4:University {name: 'ASE', country: 'RO', rank: 1000})
"""
run_query(univ)
print("Universități create!")

Universități create!


In [6]:
#CERTIFICARI
certificari = """
CREATE
  (cert1:Certification {name: 'AWS Solutions Architect', issuer: 'Amazon', difficulty: 5}),
  (cert2:Certification {name: 'CKA (Kubernetes)', issuer: 'CNCF', difficulty: 5}),
  (cert3:Certification {name: 'PMP', issuer: 'PMI', difficulty: 4})
"""
run_query(certificari)
print("Certificări create!")

Certificări create!


In [7]:
#PROIECTE & POSTĂRI
proiectePostari = """
CREATE
  (p1:Project {name: 'OpenVision AI', tech_stack: ['Python', 'PyTorch'], type: 'Open Source'}),
  (p2:Project {name: 'LearnGraph', tech_stack: ['Neo4j', 'Java'], type: 'Hackathon'}),
  (post1:Post {id: 'p001', topics: ['AI', 'Future'], timestamp: datetime({year:2024, month:3, day:10})}),
  (post2:Post {id: 'p002', topics: ['Hiring', 'Java'], timestamp: datetime({year:2024, month:3, day:12})})
"""
run_query(proiectePostari)
print("Proiecte și Postări create!")

Proiecte și Postări create!


In [8]:
#USERS
useri = """
CREATE
  (u1:User {id: 'u1', name: 'Ana Popa', role: 'Staff Engineer', languages: ['en', 'ro'], soft_skills: ['Leadership'], open_to_work: false}),
  (u2:User {id: 'u2', name: 'Bogdan Ionescu', role: 'DevOps Lead', languages: ['en', 'fr'], soft_skills: ['Problem Solving'], open_to_work: true}),
  (u3:User {id: 'u3', name: 'Cristi Stan', role: 'Security Expert', languages: ['en', 'de'], soft_skills: ['Audit'], open_to_work: false}),
  (u4:User {id: 'u4', name: 'David Munteanu', role: 'Frontend Dev', languages: ['en', 'ro'], soft_skills: ['Creativity'], open_to_work: true}),
  (u5:User {id: 'u5', name: 'Elena Radu', role: 'Junior Java Dev', languages: ['ro'], soft_skills: ['Teamwork'], open_to_work: false}),
  (u6:User {id: 'u6', name: 'Florin Dumitrescu', role: 'Fullstack Dev', languages: ['en'], soft_skills: ['Adaptability'], open_to_work: true}),
  (u7:User {id: 'u7', name: 'George Vasilescu', role: 'Product Manager', languages: ['en', 'es'], soft_skills: ['Negotiation'], open_to_work: false}),
  (u8:User {id: 'u8', name: 'Ioana Marin', role: 'HR Recruiter', languages: ['en', 'ro'], soft_skills: ['Communication'], open_to_work: false}),
  (u9:User {id: 'u9', name: 'Matei Pop', role: 'Student', languages: ['en'], soft_skills: ['Learning'], open_to_work: true}),
  (u10:User {id: 'u10', name: 'Nina Voicu', role: 'Financial Analyst', languages: ['en', 'fr'], soft_skills: ['Analysis'], open_to_work: true}),
  (u11:User {id: 'u11', name: 'Paul Serban', role: 'Developer Advocate', languages: ['en', 'ro', 'de'], soft_skills: ['Public Speaking'], open_to_work: false}),
  (u12:User {id: 'u12', name: 'Raluca Nistor', role: 'CTO', languages: ['en'], soft_skills: ['Strategy'], open_to_work: false})
"""
run_query(useri)
print("Utilizatori creați!")

Utilizatori creați!


### Relatii Noduri

In [9]:
relNoduri = """
MATCH (u1:User {id:'u1'}), (u2:User {id:'u2'}), (u3:User {id:'u3'}),
      (u4:User {id:'u4'}), (u5:User {id:'u5'}), (u6:User {id:'u6'}),
      (u7:User {id:'u7'}), (u8:User {id:'u8'}), (u9:User {id:'u9'}),
      (u10:User {id:'u10'}), (u11:User {id:'u11'}), (u12:User {id:'u12'})

MATCH (c_google:Company {name:'Google'}), (c_uipath:Company {name:'UiPath'}),
      (c_bitdef:Company {name:'Bitdefender'}), (c_bcr:Company {name:'BCR'}),
      (c_rev:Company {name:'Revolut'})

MATCH (uni_poli:University {name:'Politehnica Bucuresti'}), (uni_unibuc:University {name:'Universitatea Bucuresti'}),
      (uni_ubb:University {name:'UBB Cluj'}), (uni_ase:University {name:'ASE'})

MATCH (cert_aws:Certification {name:'AWS Solutions Architect'}), (cert_cka:Certification {name:'CKA (Kubernetes)'})

MATCH (proj_ai:Project {name:'OpenVision AI'}), (proj_edu:Project {name:'LearnGraph'})

MATCH (post_tech:Post {id:'p001'})

CREATE
  // STUDIED_AT
  (u1)-[:STUDIED_AT {degree: 'Master', year: 2015}]->(uni_poli),
  (u2)-[:STUDIED_AT {degree: 'Master', year: 2015}]->(uni_poli),
  (u5)-[:STUDIED_AT {degree: 'Bachelor', year: 2023}]->(uni_unibuc),
  (u9)-[:STUDIED_AT {degree: 'Bachelor', year: 2024}]->(uni_unibuc),
  (u3)-[:STUDIED_AT {degree: 'Bachelor', year: 2018}]->(uni_ubb),
  (u4)-[:STUDIED_AT {degree: 'Bachelor', year: 2020}]->(uni_ubb),
  (u7)-[:STUDIED_AT]->(uni_ase),
  (u10)-[:STUDIED_AT]->(uni_ase),

  // WORKS_AT
  (u1)-[:WORKS_AT {role: 'Lead', current: true}]->(c_google),
  (u11)-[:WORKS_AT {role: 'Advocate', current: true}]->(c_google),
  (u2)-[:WORKS_AT {role: 'DevOps', current: true}]->(c_uipath),
  (u6)-[:WORKS_AT {role: 'Dev', current: true}]->(c_uipath),
  (u3)-[:WORKS_AT {role: 'SecEng', current: true}]->(c_bitdef),
  (u10)-[:WORKS_AT {role: 'Analyst', current: true}]->(c_bcr),
  (u7)-[:WORKS_AT {role: 'PO', current: false, end_year: 2022}]->(c_bcr),
  (u7)-[:WORKS_AT {role: 'Senior PM', current: true}]->(c_rev),

  // Certificări & Proiecte
  (u1)-[:EARNED {year: 2022}]->(cert_aws),
  (u2)-[:EARNED {year: 2023}]->(cert_aws),
  (u3)-[:EARNED {year: 2021}]->(cert_cka),
  (u4)-[:CONTRIBUTED_TO {commits: 150}]->(proj_ai),
  (u6)-[:CONTRIBUTED_TO {commits: 80}]->(proj_ai),
  (u9)-[:CONTRIBUTED_TO {commits: 5}]->(proj_edu),
  (u11)-[:CONTRIBUTED_TO {commits: 500}]->(proj_edu),

  // FOLLOWS & Content
  (u1)-[:FOLLOWS]->(u2),
  (u2)-[:FOLLOWS]->(u3),
  (u8)-[:FOLLOWS]->(u2),
  (u12)-[:FOLLOWS]->(u1),
  (u7)-[:FOLLOWS]->(u10),
  (u4)-[:FOLLOWS]->(u11),

  (u11)-[:AUTHORED]->(post_tech),
  (u1)-[:ENGAGED_WITH {type: 'Like'}]->(post_tech),
  (u9)-[:ENGAGED_WITH {type: 'Comment'}]->(post_tech)
"""
run_query(relNoduri)
print("Relații create!")

Relații create!


## Inchidere Conexiune

In [10]:
driver.close()